In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
import random
from sklearn import tree
from sklearn.cluster import KMeans
import time 
import datetime
from datetime import date

In [2]:
%%time
pd.set_option('display.float_format', lambda x: '%.3f' % x)

df_train = pd.read_csv('BPI_Challenge_2012-training.csv')
df_test = pd.read_csv('BPI_Challenge_2012-test.csv')


unixTransform = lambda x: time.mktime(x.timetuple())
secondsTransform = lambda x: x.total_seconds()


df_train["event time:timestamp"] = pd.to_datetime(df_train["event time:timestamp"], dayfirst=True)

#df_train["UNIX"] = df_train["event time:timestamp"].apply(unixTransform)

Wall time: 1min 20s


In [3]:
# Only keep COMPLETE events - Jarl
df_complete = df_train[df_train["event lifecycle:transition"] == "COMPLETE"]

In [4]:
def EventTime(data):
    
    for i in list(data["case concept:name"].unique()):
        data.loc[data["case concept:name"] == i, "nextTIME"] = data.loc[data["case concept:name"] == i, "event time:timestamp"].shift(-1)
    return data

In [5]:
%%time
df_next = EventTime(df_complete)

C:\Users\rodri\anaconda3\lib\site-packages\pandas\core\indexing.py:1681: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.obj[key] = empty_value
C:\Users\rodri\anaconda3\lib\site-packages\pandas\core\indexing.py:1773: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_column(ilocs[0], value, pi)


Wall time: 59.2 s


In [6]:
df_next["Completion Time"] = np.nan
df_next["Completion Time"] = (df_next["nextTIME"] - df_next["event time:timestamp"]).apply(secondsTransform)

<ipython-input-6-5975738714a4>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_next["Completion Time"] = np.nan
<ipython-input-6-5975738714a4>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_next["Completion Time"] = (df_next["nextTIME"] - df_next["event time:timestamp"]).apply(secondsTransform)


In [7]:
df_next.isnull().sum()

eventID                           0
case concept:name                 0
case REG_DATE                     0
case AMOUNT_REQ                   0
event concept:name                0
event lifecycle:transition        0
event time:timestamp              0
nextTIME                      10469
Completion Time               10469
dtype: int64

In [8]:
df_next["Time Average"] = 0

<ipython-input-8-87de431324ec>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_next["Time Average"] = 0


In [9]:
for name, group in df_next[["event concept:name", "Completion Time"]].groupby(["event concept:name"]):
    mean = group["Completion Time"].mean()
    df_next.loc[df_next["event concept:name"] == name, "Time Average"] = mean
    
    
    #df_next[df_next["event concept:name"] == name]["Time Average"] = mean
    

C:\Users\rodri\anaconda3\lib\site-packages\pandas\core\indexing.py:1817: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_column(loc, value, pi)


In [10]:
df_next.groupby(["event concept:name"])[["Completion Time"]].mean()

,Completion Time
event concept:name,
A_ACCEPTED,224.205
A_ACTIVATED,3.145
A_APPROVED,51.336
A_CANCELLED,181.347
A_DECLINED,3.443
A_FINALIZED,2.314
A_PARTLYSUBMITTED,8060.300
A_PREACCEPTED,15291.844
A_REGISTERED,2.527
